# NFL Data Analysis

In this notebook, I will be examining the NFL data that was posted on the website kaggle.com. I will be looking for trends and doing some initial data analysis before doing any ML or prediction. I will record my steps and inferences so as to learn better how to analyze and pipeline datasets.

In [10]:
#Import all needed packages such as pandas and numpy
import re
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.api as sm

# Disable annoying SettingWithCopyWarning
pd.options.mode.chained_assignment = None

path = '/Users/Gerrit/Desktop/nflplaybyplay2015.csv'

#Now reading in the NFL dataset from kaggle.com using pandas
data = pd.read_csv(path)

data.head(5)

C:\Users\Gerrit\Desktop\Anaconda\envs\py35\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,...,Accepted.Penalty,PenalizedTeam,PenaltyType,PenalizedPlayer,Penalty.Yards,PosTeamScore,DefTeamScore,ScoreDiff,AbsScoreDiff,Season
0,36,2015-09-10,2015091000,1,1,NaN,15:00,15.0,3600.0,0.0,...,0,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,2015
1,51,2015-09-10,2015091000,1,1,1.0,15:00,15.0,3600.0,0.0,...,0,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,2015
2,72,2015-09-10,2015091000,1,1,1.0,14:21,15.0,3561.0,39.0,...,0,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,2015
3,101,2015-09-10,2015091000,1,1,2.0,14:04,15.0,3544.0,17.0,...,0,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,2015
4,122,2015-09-10,2015091000,1,1,1.0,13:26,14.0,3506.0,38.0,...,0,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,2015


In [11]:
data.describe()

C:\Users\Gerrit\Desktop\Anaconda\envs\py35\lib\site-packages\numpy\lib\function_base.py:3823: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,Unnamed: 0,GameID,Drive,qtr,down,TimeUnder,TimeSecs,PlayTimeDiff,yrdln,yrdline100,...,Fumble,Sack,Challenge.Replay,Accepted.Penalty,Penalty.Yards,PosTeamScore,DefTeamScore,ScoreDiff,AbsScoreDiff,Season
count,46129.000000,4.612900e+04,46129.000000,46129.000000,39006.000000,46102.000000,46102.000000,46075.000000,46021.000000,46021.000000,...,46129.000000,46129.000000,46129.000000,46129.000000,46129.000000,42878.000000,42878.000000,42878.000000,42878.000000,46129.0
mean,55648.646556,2.015164e+09,12.279607,2.583407,1.996949,7.326038,1686.735847,20.214585,28.565090,49.411312,...,0.013592,0.027401,0.008953,0.076633,0.652388,10.802066,11.864010,-1.061943,7.977797,2015.0
std,87501.612069,2.183164e+05,7.144244,1.134256,1.003834,4.659934,1065.494471,17.613538,12.591719,24.852971,...,0.115792,0.163252,0.094198,0.266011,2.716825,9.670936,10.138756,10.946834,7.570614,0.0
min,35.000000,2.015091e+09,1.000000,1.000000,1.000000,0.000000,-747.000000,0.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-41.000000,0.000000,2015.0
25%,12661.000000,2.015101e+09,6.000000,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,2015.0
50%,26032.000000,2.015111e+09,12.000000,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,2015.0
75%,39415.000000,2.015121e+09,18.000000,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,2015.0
max,466111.000000,2.016010e+09,33.000000,5.000000,4.000000,15.000000,3600.000000,940.000000,50.000000,99.000000,...,1.000000,1.000000,1.000000,1.000000,55.000000,52.000000,51.000000,41.000000,41.000000,2015.0


In [9]:
#This is the collection of column labels.
data.columns

Index([u'Unnamed: 0', u'Date', u'GameID', u'Drive', u'qtr', u'down', u'time',
       u'TimeUnder', u'TimeSecs', u'PlayTimeDiff', u'SideofField', u'yrdln',
       u'yrdline100', u'ydstogo', u'ydsnet', u'GoalToGo', u'FirstDown',
       u'posteam', u'DefensiveTeam', u'desc', u'PlayAttempted',
       u'Yards.Gained', u'sp', u'Touchdown', u'ExPointResult', u'TwoPointConv',
       u'DefTwoPoint', u'Safety', u'PlayType', u'Passer', u'PassAttempt',
       u'PassOutcome', u'PassLength', u'PassLocation', u'InterceptionThrown',
       u'Interceptor', u'Rusher', u'RushAttempt', u'RunLocation', u'RunGap',
       u'Receiver', u'Reception', u'ReturnResult', u'Returner', u'Tackler1',
       u'Tackler2', u'FieldGoalResult', u'FieldGoalDistance', u'Fumble',
       u'RecFumbTeam', u'RecFumbPlayer', u'Sack', u'Challenge.Replay',
       u'ChalReplayResult', u'Accepted.Penalty', u'PenalizedTeam',
       u'PenaltyType', u'PenalizedPlayer', u'Penalty.Yards', u'PosTeamScore',
       u'DefTeamScore', u'ScoreDif

In [13]:
#Just taking a look at the levels in each column that seems ambiguous.
# data['PlayType'].unique()
data['PlayType'].unique()

array(['Kickoff', 'Run', 'Pass', 'Sack', 'No Play', 'Field Goal', 'Punt',
       'Quarter End', 'Extra Point', 'Two Minute Warning', 'Timeout',
       'QB Kneel', 'Onside Kick', 'End of Game', 'Spike'], dtype=object)

I will create separate data sets for pass plays, run plays, kickoffs, field goal, etc.

In [36]:
#This creates datasets grouped by play type. Meaning we have all the passes, runs, kickoffs, etc.
# in different datasets. There are 15 play types
#I am also putting a filter on the data, so we are only looking at play types with more than 500 training examples.
play_types = data.groupby("PlayType")
print(type(play_types))

<class 'pandas.core.groupby.DataFrameGroupBy'>


Go through and eliminate the no plays and spikes.

In [26]:
play_types = []
for a in data['PlayType'].unique():
    play_types.append(a)
    
#Now creating different data sets by individaul play type.
pass_plays = play_
    

['Kickoff', 'Run', 'Pass', 'Sack', 'No Play', 'Field Goal', 'Punt', 'Quarter End', 'Extra Point', 'Two Minute Warning', 'Timeout', 'QB Kneel', 'Onside Kick', 'End of Game', 'Spike']


Some important questions that can be answered (?) in an NFL Dataset:
Do defenses slack off at the end of quarters?
    Relationship between end of quarters/games and successful plays
Do passes/runs have more success at beginning/end of quarters/games?
Relationship between effort and the score?
    Correlation between point differential and succcess of play
    From this, try to predict lazy or underperforming defenses.
Relationship between trying and how far into the season?
    Look for relationship between datetime and success of plays
Is there a point of no return? Meaning is there a trend showing that teams do not come back from a certain point differential at a certain time in the game?
    We can even apply this to a whole season. Is there a point in the season when a certain record will not get you into the playoffs? 

Next find the features that are real valued.
Get dummy variables for directions (left, middle, right, etc.)
Need to append all possible combinations/interactions between features.

In [ ]:
#This is a categorical scatterplot by which direction play was run
sns.swarmplot(x="", y="", hue="x_val", data=data)